## 0. Import packages

In [1]:
import pandas as pd
import gzip
import math
from tqdm import tqdm
from itertools import chain, combinations
from Association import Association
import numpy as np
import random
tqdm.pandas() #for progres_apply etc.

In [2]:
seed_index = 0

## 1. Load data from previous step

In [3]:
import pickle
rules = pickle.load(open("pickle_dumps/article_rules.p", "rb"))
user_rules = pickle.load(open("pickle_dumps/user_rules.p", "rb"))

In [4]:
df_test = pickle.load(open(f"pickle_dumps/test_df_{seed_index}.p", "rb"))
df_test = df_test.set_index("user_id").to_dict()["item_id"]

In [5]:
df_train = pickle.load(open(f"pickle_dumps/train_df_{seed_index}.p", "rb"))
df_train = df_train.set_index("user_id").to_dict()["item_id"]

In [6]:
class Recommender():
    def __init__(self, assos):
        pass
    
    def recommend(self, user_items, k, user_id=None, split=None):
        pass        

In [7]:
def hitrate(rc, k, nr_users = float("1")):
    hits = 0
    ndcg = []
    counts = 0
    avg_len = 0
    n_recall = 0
    
    
    
    user_ids = list(df_test.keys())
    for user in tqdm(random.sample(user_ids, int(len(user_ids) * nr_users))):
        user_target = df_test[user]
        user_items = df_train[user]
        
        r = rc.recommend(user_items, k, user)

        avg_len += len(r)

        correct = 0
        hit = False
        for item in user_target:
            if item in r:
                if not hit:
                    hits += 1
                    hit = True
                correct += 1
        n_recall += correct / max(1, (min(len(r), len(user_target))))

        dcg = 0
        idcg = sum(1/np.log2(i+1) for i in range(1,(max(1, len(r)))+1))
        for i, rec in enumerate(r):
            good = 1 if rec in user_target else 0
            dcg += (2**good - 1)/(np.log2(i+2))
        ndcg.append(dcg/idcg)
            
                
                
        counts += 1
    
    print("avg len: {:f}".format(avg_len / counts))
    if k == -1:
        print("HR\t{:.5f}".format((hits / counts)))
        print("EHR\t{:.5f}".format((hits / counts) * (avg_len /  counts)))
    else:
        print("HR@{}\t{:.5f}".format(k, (hits / counts)))
        print("normalised recall@{}\t{:.5f}".format(k, (n_recall / counts)))
        print("EHR@{}\t{:.5f}".format(k, (hits / counts) * (avg_len /  counts)))
    print("nDCG@{}\t{:.5f}".format(k, (sum(ndcg) / len(ndcg))))
    return {"HR": hits /counts,
           "EHR": (hits / counts) * (avg_len /  counts),
           "normalised recall": (n_recall / counts),
           "nDCG": sum(ndcg) / len(ndcg)}

In [8]:
class Assos_Recommender(Recommender):
    def __init__(self, assos, sorter=lambda x: x.s * x.c):
        # use popular items to fill unused space
        # TODO (df_train is used for this)
        
        self.sorter = sorter
        self.assos = sorted(assos, key=self.sorter, reverse=True)[:100000]
        
#     def get_items(self, user_id):
#         return self.df_test[user_id]
    
    def recommend(self, user_items, k, user_id=None, split=None):
        recommendation = set()
        user_items = set(user_items)
        for a in self.assos:
#             print(a.left, user_items)
            stop = False
            for l in a.left:
                # break if any element is not in the user session
                if l not in user_items:
                    stop = True
                    break
                    
                # all elements are in the user session --> add RHS of rule to the recommendation
            if not stop:
                r = a.right
                for item in r:
                    if item not in user_items:
                        recommendation.add(item)
                        if len(recommendation) >= k and k != -1:
                            break
                        
            if len(recommendation) >= k and k != -1:
                break
                
        return recommendation

# Results
rule.s --> 18.317 <br>
rule.s * rule.c --> 20.545

In [9]:
results = []

In [14]:
ar = Assos_Recommender(rules, lambda x: x.z)
results.append(("lambda x: x.z", hitrate(ar, 10, 0.1)))

100%|██████████| 6297/6297 [01:36<00:00, 65.44it/s]

avg len: 6.254407
HR@10	0.24631
normalised recall@10	0.06225
EHR@10	1.54051
nDCG@10	0.05166


In [15]:
ar = Assos_Recommender(rules, lambda x: x.s * x.c)
results.append(("lambda x: x.s * x.c", hitrate(ar, 10, 0.1)))

100%|██████████| 6297/6297 [04:46<00:00, 22.00it/s]

avg len: 5.952676
HR@10	0.38923
normalised recall@10	0.11703
EHR@10	2.31698
nDCG@10	0.08194


In [16]:
ar = Assos_Recommender(rules, lambda x: x.s + x.c)
results.append(("lambda x: x.s + x.c", hitrate(ar, 10, 0.1)))

  2%|▏         | 137/6297 [00:02<01:50, 55.75it/s]


KeyboardInterrupt: 

In [ ]:
ar = Assos_Recommender(rules, lambda x: x.s)
results.append(("lambda x: x.s", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: x.c)
results.append(("lambda x: x.c", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: x.lift)
results.append(("lambda x: x.lift", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: -x.lift)
results.append(("lambda x: -x.lift", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: 2 * x.s + x.c)
results.append(("lambda x: 2 * x.s + x.c", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: x.s * x.s * x.c)
results.append(("lambda x: x.s * x.s * x.c", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: np.sqrt(np.power(x.s, 2) + np.power(x.c, 2)))
results.append(("lambda x: np.sqrt(np.power(x.s, 2) + np.power(x.c, 2))", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: np.sqrt(2 * np.power(x.s, 2) + np.power(x.c, 2)))
results.append(("lambda x: np.sqrt(2 * np.power(x.s, 2) + np.power(x.c, 2))", hitrate(ar, 10, 0.1))

In [ ]:
ar = Assos_Recommender(rules, lambda x: -np.abs(x.lift - 1))
results.append(("lambda x: -np.abs(x.lift - 1)", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: np.abs(x.lift - 1))
results.append(("lambda x: np.abs(x.lift - 1)", hitrate(ar, 10, 0.1)))

In [ ]:
ar = Assos_Recommender(rules, lambda x: x.lift * x.s * x.c)
results.append(("lambda x: x.lift * x.s * x.c", hitrate(ar, 10, 0.1)))

In [17]:
class User_Recommender(Recommender):
    def __init__(self, assos):
        self.assos = assos
    
    def recommend(self, user_items, k, user_id):
        if len(user_items) <= 0 or user_id not in self.assos:
            return set()
        
        user_items = set(user_items)
        rdict = dict()

        for a in self.assos[user_id]:
            item_list = list()
            for l in a.left:
                tmp = []
                tmp.extend(df_test[l])
                tmp.extend(df_train[l])
                item_list.append(set(tmp))

#                     if l in df_test:
#                         item_list.append(set(df_test[l]))
#                     if l in df_train:
#                         item_list.append(set(df_train[l]))

            if item_list:
                intersection = set.intersection(*item_list)
                for item in intersection - user_items:
                    if item in rdict:
                        rdict[item] += (a.c * a.s)
                    else:
                        rdict[item] = (a.c * a.s)

                            
#         if rdict.values():   
#             if max(rdict.values()) < 50:
#                 return set()
            
        rlist = sorted(rdict.keys(), key=lambda x: rdict[x], reverse=True)

        recommendation = set()
        for i in rlist:
            recommendation.add(i)
            if len(recommendation) >= k and k != -1:
                break
        return recommendation

# Results
sum(rule.s * rule.c) --> 23.020

In [19]:
ur = User_Recommender(user_rules)
hr = hitrate(ur, 10, 0.1)

100%|██████████| 6297/6297 [00:19<00:00, 315.56it/s] 

avg len: 0.190567
HR@10	0.01874
normalised recall@10	0.00659
EHR@10	0.00357
nDCG@10	0.00660


In [14]:
class Combo_Recommender(Recommender):
    def __init__(self, ar, ur):
        self.ar = ar
        self.ur = ur
        
    def recommend(self, user_items, k, user_id):
        recommendation = self.ur.recommend(user_items, k, user_id)
        if len(recommendation) < k:
            recommendation.update(self.ar.recommend(user_items, k, user_id))
        return recommendation

no filter: 36139; 08554 <br>
25: idem<br>

In [15]:
cr = Combo_Recommender(ar, ur)
hr = hitrate(cr, 10)

100%|██████████| 62970/62970 [04:09<00:00, 252.48it/s]

avg len: 9.971780
HR@10	0.66540
normalised recall@10	0.25960
EHR@10	6.63518
nDCG@10	0.13109


In [ ]:
class Combo_Recommender2(Recommender):
    def __init__(self, ar, ur):
        self.ar = ar
        self.ur = ur
        
    def recommend(self, user_items, k, user_id, split):
        recommendation = self.ur.recommend(user_items, 8, user_id, split)
        if len(recommendation) < k:
            recommendation.update(self.ar.recommend(user_items, k - len(recommendation), user_id, split))
        return recommendation

In [ ]:
cr = Combo_Recommender2(ar, ur)
hr = hitrate(cr, 10)

In [ ]:
class Combo_Recommender3(Recommender):
    def __init__(self, ar, ur):
        self.ar = ar
        self.ur = ur
        
    def recommend(self, user_items, k, user_id, split):
        recommendation = self.ar.recommend(user_items, 5, user_id, split)
        if len(recommendation) < k:
            recommendation.update(self.ur.recommend(user_items, k - len(recommendation), user_id, split))
        return recommendation

In [ ]:
cr = Combo_Recommender3(ar, ur)
hr = hitrate(cr, 10)